In [18]:
import logging
import os
import re
import shutil
import subprocess
import sys
import textwrap
import urllib.parse

import progressbar
import toml

from typing import Tuple, Callable, Type, Any

logging.basicConfig(level=logging.INFO, format='%(asctime)s:%(threadName)s:%(message)s')

In [37]:
# Path to store package registry and the Git executable
packages_path = r'K:\julia_packages'
git = r'K:\Git\bin'
gnu_tools = r'K:\GnuUtils\bin'

os.environ['PATH'] = os.pathsep.join([git, gnu_tools] + os.environ['PATH'].split(os.pathsep))

In [38]:
# Set the URL to the Julia default package registry and clone/fetch the latest registry
reg_url = 'https://github.com/JuliaRegistries/General.git'
reg_path = os.path.join(packages_path, 'General')
if os.path.exists(reg_path):
    subprocess.run(['git', 'pull'], cwd=reg_path)
else:
    subprocess.run(['git', 'clone', reg_url, reg_path])

In [39]:
# Parse the registry TOML file into a really big dictionary
with open(os.path.join(reg_path, 'Registry.toml')) as infile:
    registry_data = toml.load(infile)

In [40]:
# Walk through the TOML dictionary, grabbing the package names and their Git repo URLs
pkg_download = {}
for pkg in sorted(registry_data['packages'].values(), key=lambda x: x['name']):
    with open(os.path.join(reg_path, pkg['path'], 'Package.toml')) as infile:
        pkg_toml = toml.load(infile)
    url = urllib.parse.urlparse(pkg_toml['repo'])
    url = url._replace(scheme='https', netloc=':@' + url.netloc)
    pkg_repo = url.geturl()
    pkg_download[pkg_toml['name']] = pkg_repo

In [41]:
# Function to run Git with timeout protection
def run(*args, **kwargs):
    success = False
    while not success:
        try:
            print(f"Running {' '.join([f'{val}' for val in args] + [f'{key}={val}' for key, val in kwargs.items()])}",
                  file=sys.__stdout__)
            subprocess.run(args, **kwargs)
            success = True
        except subprocess.TimeoutExpired:
            print('*** Rerunning! ***', file=sys.__stdout__)

In [42]:
# Loop over the package URLs, cloning or fetching on each one
for pkg_repo in progressbar.progressbar(pkg_download.values()):
    pkg_repo_base = os.path.basename(pkg_repo)
    pkg_path = os.path.join(packages_path, pkg_repo_base)
    if os.path.exists(pkg_path):
        run('git', 'fetch', cwd=pkg_path, timeout=10)
    else:
        run('git', 'clone', '--mirror', pkg_repo, pkg_path, timeout=60)

100% (2721 of 2721) |####################| Elapsed Time: 3:19:46 Time:  3:19:46


In [43]:
def onerror(func: Callable, path: str, exc_info: Tuple[Type, Exception, Any]):
    
    """Error handler for ``shutil.rmtree``.

    If the error is due to an access error (read only file)
    it attempts to add write permission and then retries.

    If the error is for another reason it re-raises the error.

    Usage : ``shutil.rmtree(path, onerror=onerror)``
    
    """
    
    import stat
    if not os.access(path, os.W_OK):
        # Is the error an access error ?
        os.chmod(path, stat.S_IWUSR)
        func(path)
    else:
        raise

In [ ]:
# The Julia language repo needs to download some dependencies when it's built.
# This is normally done using "make -d deps getall", but we don't have GNU make
# on the IAS. So we have to walk the Makefiles ourselves and manuall download
# the dependencies so we can ingress them, too!

# Clone Julia from the bare repo
julia_repo = os.path.join(packages_path, 'julia.git')
julia_path = os.path.join(packages_path, 'julia')
if os.path.exists(julia_path):
    shutil.rmtree(julia_path, onerror=onerror)
run('git', 'clone', julia_repo, julia_path)

In [ ]:
# Run GNU make to get the dependencies
subprocess.run(['make', 'getall'], cwd=os.path.join(julia_path, 'deps'))

In [ ]:
# Zip up the directory to transfer to NNPP
topdir = os.path.realpath(os.path.dirname(packages_path))
basename = os.path.basename(packages_path)
subprocess.run(['tar', 'czf', f'{basename}.tar.gz', basename], cwd=topdir)